In [119]:
import numpy as np
from mpcpp import MPC_library as mpc
from astropy import units as u
from astropy.time import Time
from astropy.coordinates import EarthLocation, solar_system_ephemeris
from astropy.coordinates import get_body_barycentric, get_body
from jplephem.spk import SPK
from importlib import reload
reload(mpc)

<module 'mpcpp.MPC_library' from '/home/malexand/.anaconda3/envs/astroconda36/lib/python3.6/mpcpp/MPC_library.py'>

In [120]:
def mpc_xyz(time=2458892.0, obsCode='I11'):
    ''' MPC ephemeris '''
    obs = mpc.Observatory()

    print('MPC XYZ:')
    
    ''' Heliocentric location of geocenter: '''
    helio_500_mpc = obs.getObservatoryPosition(obsCode='500', jd_utc=time, old=False) * u.au
    print('Heliocentric location of  geocenter:  {}'.format(helio_500_mpc.to(u.km)))

    ''' Heliocentric location of observatory: '''
    helio_OBS_mpc = obs.getObservatoryPosition(obsCode=obsCode, jd_utc=time, old=False) * u.au
        
    ''' Geocentric location of observatory '''
    geo_OBS_mpc = helio_OBS_mpc - helio_500_mpc
    print(' Geocentric  location of observatory: {}'.format(geo_OBS_mpc.to(u.km)))

    print('Heliocentric location of observatory: {}\n'.format(helio_OBS_mpc.to(u.km)))
        
    return helio_500_mpc, geo_OBS_mpc, helio_OBS_mpc

In [121]:
def apy_xyz(time=2458892.0, site_name='gemini_south'):
    ''' Astropy ephemeris '''
    print('Astropy XYZ:')
    t_jd = Time(time, format='jd')
    
    ''' Barycentric location of geocenter: '''
    bary_500_apy = get_body_barycentric('earth', t_jd, ephemeris='de430').xyz

    ''' Barycentric location of the Sun: '''
    bary_Sun_apy = get_body_barycentric('sun', t_jd, ephemeris='de430').xyz

    ''' Heliocentric location of geocenter: '''
    helio_500_apy = bary_500_apy - bary_Sun_apy
    print('Heliocentric location of  geocenter:  {}'.format(helio_500_apy))

    ''' Geocentric location of observatory: '''
    loc_0 = EarthLocation.of_site(site_name)        # at reference time
    loc_t, _ = loc_0.get_gcrs_posvel(obstime=t_jd)  # at correct time
    geo_OBS_apy = loc_t.xyz                         # in km, not AU
    print(' Geocentric  location of observatory: {}'.format(geo_OBS_apy))

    ''' Heliocentric location of observatory: '''
    helio_OBS_apy = helio_500_apy + geo_OBS_apy
    print('Heliocentric location of observatory: {}\n'.format(helio_OBS_apy))
    
    return helio_500_apy, geo_OBS_apy, helio_OBS_apy

In [122]:
def diff_apy_mpc(time=2458892.0, obsCode='I11', site_name='gemini_south'):
    ''' Comparing Astropy and MPC '''
    print('v'*30, site_name, obsCode, 'v'*30)

    helio_500_apy, geo_OBS_apy, helio_OBS_apy = apy_xyz(time, site_name)
    helio_500_mpc, geo_OBS_mpc, helio_OBS_mpc = mpc_xyz(time, obsCode)

    print('Discrepancies (Astropy - MPC):')
    helio_500_diff = helio_500_apy - helio_500_mpc
    helio_500_diff_abs = (np.sum(helio_500_diff ** 2) ** 0.5).to(u.m)
    print('Heliocentric location of  geocenter:  {0:}'.format(helio_500_diff_abs))

    geo_OBS_diff = geo_OBS_apy - geo_OBS_mpc
    geo_OBS_diff_abs = (np.sum(geo_OBS_diff ** 2) ** 0.5).to(u.m)
    print(' Geocentric  location of observatory: {0:}'.format(geo_OBS_diff_abs))

    helio_OBS_diff = helio_OBS_apy - helio_OBS_mpc
    helio_OBS_diff_abs = (np.sum(helio_OBS_diff ** 2) ** 0.5).to(u.m)
    print('Heliocentric location of observatory: {0:}'.format(helio_OBS_diff_abs))
    
    print('^'*30, site_name, obsCode, '^'*30, '\n')
    
    return helio_500_apy, geo_OBS_apy, helio_OBS_apy, helio_500_mpc, geo_OBS_mpc, helio_OBS_mpc

In [178]:
time_jd = 2458892.0
helio_500_apy, geo_OBS_apy, helio_OBS_apy, helio_500_mpc, geo_OBS_mpc, helio_OBS_mpc = diff_apy_mpc(time_jd, 'I11', 'gemini_south')
helio_500_apy, geo_OBS_apy, helio_OBS_apy, helio_500_mpc, geo_OBS_mpc, helio_OBS_mpc = diff_apy_mpc(time_jd, 'T14', 'CFHT')  # Mauna Kea-UH/Tholen NEO Follow-Up (CFHT)
helio_500_apy, geo_OBS_apy, helio_OBS_apy, helio_500_mpc, geo_OBS_mpc, helio_OBS_mpc = diff_apy_mpc(time_jd, 'T09', 'Subaru')  # Mauna Kea-UH/Tholen NEO Follow-Up (Subaru)
helio_500_apy, geo_OBS_apy, helio_OBS_apy, helio_500_mpc, geo_OBS_mpc, helio_OBS_mpc = diff_apy_mpc(time_jd, 'G83', 'Large Binocular Telescope')  # Mt. Graham-LBT
helio_500_apy, geo_OBS_apy, helio_OBS_apy, helio_500_mpc, geo_OBS_mpc, helio_OBS_mpc = diff_apy_mpc(time_jd-182, 'G83', 'Large Binocular Telescope')  # Mt. Graham-LBT
helio_500_apy, geo_OBS_apy, helio_OBS_apy, helio_500_mpc, geo_OBS_mpc, helio_OBS_mpc = diff_apy_mpc(time_jd-91, 'G83', 'Large Binocular Telescope')  # Mt. Graham-LBT

vvvvvvvvvvvvvvvvvvvvvvvvvvvvvv gemini_south I11 vvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
JPL XYZ:
Heliocentric location of  geocenter:  [-1.17854006e+08  8.16169879e+07  3.53807465e+07] km
 Geocentric  location of observatory: [-1800873.36845069 -5217154.39563667 -3191412.49208819] m
Heliocentric location of observatory: [-1.17855807e+08  8.16117707e+07  3.53775551e+07] km

MPC XYZ:
Heliocentric location of  geocenter:  [-1.17854006e+08  8.16169879e+07  3.53807465e+07] km
 Geocentric  location of observatory: [-1800.85214402 -5217.13544829 -3191.38491209] km
Heliocentric location of observatory: [-1.17855807e+08  8.16117708e+07  3.53775551e+07] km

Discrepancies (Astropy - MPC):
Heliocentric location of  geocenter:  0.057528397285059554 m
 Geocentric  location of observatory: 39.62492005329099 m
Heliocentric location of observatory: 39.57350578093613 m
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ gemini_south I11 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ 

vvvvvvvvvvvvvvvvvvvvvvvvvvvvvv Subaru T09 vvvvvvvvvvvvvvvvvvvvvv

In [179]:
print(helio_500_mpc)

[0.62900614 0.70095122 0.30386156] AU


In [189]:
#EarthLocation.get_site_names()

In [186]:
'''geocentric location of observatory'''
geo_OBS_horizons = [1.800856351642355E+03, 5.217132743849016E+03, 3.191382310369976E+03] * u.km
#geo_OBS_horizons = [3.897755155906439E-05, -9.519648682208122E-06, -1.445320321166943E-05] * u.au
geo_OBS_horizons = [3.897607996384405E-05, -9.524410913145084E-06, -1.445330040715949E-05] * u.au
geo_OBS_horizons = [3.047176273153229E-05, 1.896779432438119E-05, -2.297248162195744E-05] * u.au
geo_OBS_horizons = [-3.035225384512615E-05, -1.929568746151384E-05,-2.285733119326494E-05] * u.au
geo_OBS_horizons = [1.908870978408335E-05, -3.041274366209686E-05, -2.295052590013195E-05] * u.au
'''Astropy-Horizons'''
print((geo_OBS_apy + geo_OBS_horizons).to(u.m))
#print((np.sum((geo_OBS_apy + geo_OBS_horizons) ** 2) ** 0.5).to(u.m))
'''MPC-Horizons'''
print((geo_OBS_mpc + geo_OBS_horizons).to(u.m))
#print((np.sum((geo_OBS_mpc + geo_OBS_horizons) ** 2) ** 0.5).to(u.m))

[-1276.04811281 -1090.09543101  -142.71298803] m
[ 1.27040217 -0.12413123 -5.01807926] m


In [187]:
'''heliocentric location of geocentre'''
helio_500_horizons = [7.878053705097503E-01, -5.455758662127957E-01, -2.365056822060531E-01] * u.au
'''Astropy-Horizons:'''
print((helio_500_apy + helio_500_horizons).to(u.m))
'''MPC-Horizons:'''
print((helio_500_mpc + helio_500_horizons).to(u.m))

[2.11951985e+11 2.32438214e+10 1.00762963e+10] m
[2.11951985e+11 2.32438214e+10 1.00762963e+10] m


In [188]:
'''heliocentric location of observer'''
helio_OBS_horizons = [7.878174084909276E-01, -5.455409918344410E-01, -2.364843491329590E-01] * u.au
#helio_OBS_horizons = [7.878443480613093E-01, -5.455853858614780E-01, -2.365201354092648E-01] * u.au
helio_OBS_horizons = [7.878443465897141E-01, -5.455853906237088E-01, -2.365201355064603E-01] * u.au
helio_OBS_horizons = [7.878358422724819E-01, -5.455568984184713E-01, -2.365286546876751E-01] * u.au
helio_OBS_horizons = [-7.890835808625358E-01, 5.828815097032926E-01, 2.526647001858481E-01] * u.au
helio_OBS_horizons = [-6.289870474582483E-01, -7.009816282331524E-01, -3.038845132122471E-01] * u.au
'''Astropy-Horizons:'''
print((helio_OBS_apy + helio_OBS_horizons).to(u.m))
'''MPC-Horizons:'''
print((helio_OBS_mpc + helio_OBS_horizons).to(u.m))

[-1275.68267286 -1090.36818147  -142.83219725] m
[ 1.27769069 -0.1334509  -5.02311033] m


In [162]:
print((helio_500_apy + geo_OBS_mpc + helio_OBS_horizons).to(u.m))

[10.54319739 -0.331074   -4.63458151] m


In [130]:
kernel = SPK.open('/home/malexand/Github/mpcpp/de430.bsp')
def jpl_xyz(time=2458892.0):
    ''' JPL ephemeris '''
    print('JPL XYZ:')
    t_jd = Time(time, format='jd')
    
    ''' Barycentric location of geocenter: '''
    bary_500_JPL = kernel[0, 3].compute(time) + kernel[3, 399].compute(time)

    ''' Barycentric location of the Sun: '''
    bary_Sun_JPL = kernel[0, 10].compute(time)

    ''' Heliocentric location of geocenter: '''
    helio_500_JPL = bary_500_JPL - bary_Sun_JPL
    print('Heliocentric location of  geocenter:  {}'.format(helio_500_JPL))

    return helio_500_JPL

In [190]:
helio_500_JPL = jpl_xyz(2458892.000800753)
#helio_500_JPL = jpl_xyz(2458892.0)
print(helio_500_apy-helio_500_JPL*u.km)
print((helio_500_JPL*u.km + geo_OBS_mpc + helio_OBS_horizons).to(u.m))
print((helio_500_JPL*u.km + helio_500_horizons))
print((helio_500_mpc + helio_500_horizons).to(u.m))

JPL XYZ:
Heliocentric location of  geocenter:  [-1.17854006e+08  8.16169879e+07  3.53807465e+07]
[2.11951985e+08 2.32438214e+07 1.00762963e+07] km
[-2.11951985e+11 -2.32438214e+10 -1.00762963e+10] m
[ 1.11758709e-06 -1.55270100e-05 -7.24941492e-06] km
[2.11951985e+11 2.32438214e+10 1.00762963e+10] m


In [132]:
print((helio_500_JPL*u.km - helio_500_mpc).to(u.m))

[ 0.00000000e+00 -1.49011612e-05  0.00000000e+00] m


In [184]:
print(np.sum(np.array([3.31129314, -4.00676584, 3.9428182])**2)**0.5)
print(np.sum(np.array([4.20874424, -2.71995698, -2.60897765])**2)**0.5)
print(np.sum(np.array([10.50804217, -0.37286532, -4.6526948])**2)**0.5)
print(np.sum(np.array([0.11250733, 1.26624729, -5.03444992])**2)**0.5)
print(np.sum(np.array([7.02117847, -4.24853869, -6.82875837])**2)**0.5)

6.524158958352655
5.6496423275268866
11.498067128665758
5.192468208446161
10.67609333903856
